# KPI and Licensing Dashboard

Search BMC Helix Discovery for top-level node counts, compare against KPI targets, and estimate license Resource Units (RUs).

- Loads Discovery target and token from `../config.yaml`
- Exports CSVs and PNGs under `../output_<appliance>/kpi_licensing/`
- RU rules applied (rounded up): Servers=1 RU; Network Devices=5:1; Storage Ports=5:1; Clients=5:1; Containers=20:1; Cloud/PaaS=5:1

In [ ]:
# Environment setup
import os, sys, json, math, pathlib, datetime
from typing import Optional, Dict, List

# Ensure repo root on sys.path (robust in notebook or script)
try:
    NOTEBOOK_DIR = pathlib.Path(__file__).resolve().parent
except NameError:
    # In Jupyter, __file__ is undefined; fall back to CWD
    NOTEBOOK_DIR = pathlib.Path.cwd().resolve()
# If running from repo root, use it; else assume we're in notebooks/
REPO_ROOT = NOTEBOOK_DIR if (NOTEBOOK_DIR / 'config.yaml').exists() else NOTEBOOK_DIR.parent
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

print('Repo root:', REPO_ROOT)

In [ ]:
# Config: select appliance and optional KPI targets
import yaml

CONFIG_PATH = REPO_ROOT / 'config.yaml'
with open(CONFIG_PATH, 'r') as f:
    CFG = yaml.safe_load(f)

# Choose appliance by name from config.yaml
APPLIANCE_NAME = 'instance-prod'  # change to 'dev' or another name present in config.yaml
# Optional: override license threshold here (takes precedence over config). Example: 5000
LICENSE_THRESHOLD_OVERRIDE = 5000

# KPI targets (user-provided). Set numeric targets or leave as None/0 to skip % calc.
kpi_targets = {
    # 'Display Name': target_value
    'Hosts': 5000,
    'Windows Desktops': 10000,
    'Network Devices': None,
    'Software Instances': None,
    'Virtual Machines': None,
    'Storage Ports': None,
    'Containers/Pods': None,
    'Cloud/PaaS Resources': None,
}

# Optional: custom attribute-based KPIs (each entry counted independently)
# Example: {'name': 'Hosts: Windows Servers', 'node': 'Host', 'where': "os_type matches 'Windows' and not (type matches 'Windows Desktop' or type matches 'Laptop')", 'target': 1000}
custom_kpi_filters: List[Dict[str, Optional[str]]] = [
    {'name': 'Hosts: Windows Desktops', 'node': 'Host', 'where': "host_type has subword 'desktop' or host_type has subword 'laptop'", 'target': None},
]

print("Ready.")

In [ ]:
# Connect to Discovery (using core.access and core.api)
from core import access, api
import tideway

# Resolve appliance from config
appliances = {a['name']: a for a in CFG.get('appliances', [])}
assert APPLIANCE_NAME in appliances, f"Appliance '{APPLIANCE_NAME}' not found in config.yaml"
ap = appliances[APPLIANCE_NAME]
target = ap.get('target')
token_inline = ap.get('token')
token_file = ap.get('token_file')
license_threshold = ap.get('license_threshold', CFG.get('license_threshold'))
# Apply notebook override if provided
if 'LICENSE_THRESHOLD_OVERRIDE' in globals() and LICENSE_THRESHOLD_OVERRIDE not in (None, '', 'None'):
    license_threshold = LICENSE_THRESHOLD_OVERRIDE
# Resolve token_file relative to repo root if it's a relative path
if token_file:
    p = pathlib.Path(token_file).expanduser()
    token_file = str(p if p.is_absolute() else (REPO_ROOT / p).resolve())

class Args:
    def __init__(self, target, token_inline, token_file):
        self.target = target
        # Prefer inline token from config when present
        self.token = token_inline
        self.f_token = token_file
        # Unused but expected by some helpers
        self.output_file = None

args = Args(target, token_inline, token_file)

# Create appliance and endpoints
disco = access.api_target(args)
disco_ep, search_ep, creds_ep, vault_ep, knowledge_ep = api.init_endpoints(disco, args)

print('Connected to:', target)

In [ ]:
# Helpers: safe counting and exports
import pandas as pd
DEBUG_COUNTS = True  # set to False to silence debug

def _count_query(node: str, where: Optional[str] = None) -> str:
    q = f"search {node}"
    if where:
        q += f" where {where}"
    # Return only an aggregate count to avoid row limits
    q += " process with unique(0)"
    return q

def get_count(search, node: str, where: Optional[str] = None) -> int:
    """Return the count for a node class (optionally with a where-clause).
    Uses format='object' with an aggregate 'count' column.
    Returns 0 on any error.
    """
    try:
        q = _count_query(node, where)
        if DEBUG_COUNTS:
            print(f"[debug] query => {q}")
        # Pass the raw query string; api.get_json returns list/dict for non-response objects
        resp = search.search(q, format='object', limit=0)
        data = api.get_json(resp)
        if DEBUG_COUNTS:
            print(f"[debug] type={type(data).__name__}, len={len(data) if hasattr(data,'__len__') else 'n/a'}")
            if isinstance(data, list) and data:
                print(f"[debug] first keys => {list(data[0].keys())}")
                print(f"[debug] first row => {data[0]}")
            else:
                print(f"[debug] data => {data}")
        if isinstance(data, list):
            # Count returned rows (after any 'process with unique(0)' dedupe)
            return int(len(data))
    except Exception as e:
        if DEBUG_COUNTS:
            print(f"[debug] get_count error for node={node}: {e}")
    return 0

def get_first_found_count(search, nodes: List[str], where: Optional[str] = None) -> int:
    """Try each node class and return the first non-zero count (or last zero).
    Useful where environments differ in taxonomy names.
    """
    result = 0
    for n in nodes:
        c = get_count(search, n, where)
        if DEBUG_COUNTS:
            print(f"[debug] tried node={n}, count={c}")
        if c:
            return c
        result = c
    return result

def ensure_output_dir(base_root: pathlib.Path, host: str) -> pathlib.Path:
    safe_host = host.replace('/', '_').replace('\\', '_').replace(':', '_').replace('.', '_')
    outdir = base_root / f'output_{safe_host}' / 'kpi_licensing'
    outdir.mkdir(parents=True, exist_ok=True)
    return outdir

out_dir = ensure_output_dir(REPO_ROOT, target)
print('Output dir:', out_dir)

In [ ]:
# 1) Top-level counts
top_nodes = [
    'Host',
    'NetworkDevice',
    'Printer',
    'StorageSystem',
    'StorageDevice',
    'SoftwareInstance',
    'VirtualMachine',
    'SNMPManagedDevice',
    'ManagementController',
]

counts = []
for node in top_nodes:
    c = get_count(search_ep, node)
    counts.append({'Node': node, 'Count': c})

# Extended counts for licensing-related objects that might not be in 'top_nodes'
count_storage_ports = get_first_found_count(search_ep, ['StoragePort'])
count_containers = get_first_found_count(search_ep, ['SoftwarePod', 'SoftwareContainer'])
count_cloud = get_first_found_count(search_ep, ['CloudResource', 'CloudService'])

counts_df = pd.DataFrame(counts).sort_values('Node').reset_index(drop=True)
counts_df.loc[len(counts_df)] = {'Node': 'StoragePort', 'Count': count_storage_ports}
counts_df.loc[len(counts_df)] = {'Node': 'Containers/Pods', 'Count': count_containers}
counts_df.loc[len(counts_df)] = {'Node': 'Cloud/PaaS', 'Count': count_cloud}

display(counts_df)
counts_df.to_csv(out_dir / 'node_counts.csv', index=False)
print('Saved:', out_dir / 'node_counts.csv')

In [ ]:
# 2) KPI computations (Targets vs Discovered)
# Baseline discovered values
discovered = {
    'Hosts': get_count(search_ep, 'Host','not (host_type has subword \'desktop\' or host_type has subword \'client\') and not (cloud and nodecount(traverse ContainedHost:HostContainment:HostContainer:VirtualMachine where cloud)) and (nodecount(traverse InferredElement:Inference::DiscoveryAccess))'),
    'Windows Desktops': get_count(search_ep, 'Host', "host_type has subword 'desktop' or host_type has subword 'laptop'"),
    'Network Devices': get_count(search_ep, 'NetworkDevice'),
    'Software Instances': get_count(search_ep, 'SoftwareInstance'),
    'Virtual Machines': get_count(search_ep, 'VirtualMachine'),
    'Storage Ports': count_storage_ports,
    'Containers/Pods': count_containers,
    'Cloud/PaaS Resources': get_count(search_ep,'SoftwareInstance,VirtualMachine','__resource_unit = \'cloud\''),
}

rows = []
for name, target in kpi_targets.items():
    disc = int(discovered.get(name, 0) or 0)
    tgt = (None if target in (None, '') else int(target))
    pct = (None if not tgt or tgt <= 0 else round(100.0 * disc / tgt, 2))
    shortfall = (None if tgt is None else max(0, tgt - disc))
    rows.append({'KPI': name, 'Target': tgt, 'Discovered': disc, 'Achieved_%': pct, 'Shortfall': shortfall})

# Custom attribute KPIs
for spec in custom_kpi_filters:
    if not spec:
        continue
    name = spec.get('name') or f"{spec.get('node','?')} filter"
    node = spec.get('node') or 'Host'
    where = spec.get('where')
    target = spec.get('target')
    disc = get_count(search_ep, node, where)
    tgt = (None if target in (None, '') else int(target))
    pct = (None if not tgt or tgt <= 0 else round(100.0 * disc / tgt, 2))
    shortfall = (None if tgt is None else max(0, tgt - disc))
    rows.append({'KPI': name, 'Target': tgt, 'Discovered': disc, 'Achieved_%': pct, 'Shortfall': shortfall})

kpi_df = pd.DataFrame(rows)
display(kpi_df)
kpi_df.to_csv(out_dir / 'kpi_targets.csv', index=False)
print('Saved:', out_dir / 'kpi_targets.csv')

In [ ]:
# 3) Licensing RU estimates (rounded up)
from math import ceil

# Servers = Hosts excluding clients (desktops/laptops)
total_hosts = discovered['Hosts']
client_hosts = discovered['Windows Desktops']
servers = max(0, int(total_hosts) - int(client_hosts))

ru_breakdown = {
    'Servers': int(servers),                                 # 1 RU per server
    'Network Devices': int(ceil(discovered['Network Devices'] / 5.0)),
    'Storage Ports': int(ceil(discovered['Storage Ports'] / 5.0)),
    'Clients (Desktops/Laptops)': int(ceil(discovered['Windows Desktops'] / 5.0)),
    'Containers/Pods': int(ceil(discovered['Containers/Pods'] / 20.0)),
    'Cloud/PaaS Resources': int(ceil(discovered['Cloud/PaaS Resources'] / 5.0)),
}

ru_df = pd.DataFrame([{'Category': k, 'Count':
                          (servers if k=='Servers' else
                           discovered['Network Devices'] if k=='Network Devices' else
                           discovered['Storage Ports'] if k=='Storage Ports' else
                           discovered['Windows Desktops'] if k=='Clients (Desktops/Laptops)' else
                           discovered['Containers/Pods'] if k=='Containers/Pods' else
                           discovered['Cloud/PaaS Resources']) ,
                      'RUs': v} for k, v in ru_breakdown.items()])
ru_df.loc[len(ru_df)] = {'Category': 'TOTAL', 'Count': ru_df['Count'].sum(), 'RUs': ru_df['RUs'].sum()}
display(ru_df)
ru_df.to_csv(out_dir / 'ru_breakdown.csv', index=False)
print('Saved:', out_dir / 'ru_breakdown.csv')

In [ ]:
# 4) Charts: KPIs and RU consumption
import matplotlib.pyplot as plt
%matplotlib inline

# KPI Target vs Discovered (only where target provided)
kpi_plot = kpi_df.dropna(subset=['Target']).copy()
if not kpi_plot.empty:
    fig, ax = plt.subplots(figsize=(10, 5))
    x = range(len(kpi_plot))
    ax.bar([i-0.2 for i in x], kpi_plot['Target'], width=0.4, label='Target')
    ax.bar([i+0.2 for i in x], kpi_plot['Discovered'], width=0.4, label='Discovered')
    ax.set_xticks(list(x))
    ax.set_xticklabels(kpi_plot['KPI'], rotation=45, ha='right')
    ax.set_ylabel('Count')
    ax.set_title('KPI Targets vs Discovered')
    ax.legend()
    plt.tight_layout()
    kpi_png = out_dir / 'kpi_targets.png'
    plt.savefig(kpi_png, dpi=150)
    print('Saved:', kpi_png)
    plt.show()
else:
    print('No KPI targets provided; skipping KPI chart.')

# RU breakdown chart
ru_plot = ru_df[ru_df['Category'] != 'TOTAL']
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(ru_plot['Category'], ru_plot['RUs'])
# Optional: draw a horizontal license threshold if provided (config.yaml: license_threshold)
RU_THRESHOLD = None
try:
    RU_THRESHOLD = int(license_threshold) if license_threshold not in (None, '', 'None') else None
except Exception:
    RU_THRESHOLD = None
if RU_THRESHOLD is not None:
    ymin, ymax = ax.get_ylim()
    if RU_THRESHOLD > ymax:
        ax.set_ylim(ymin, RU_THRESHOLD * 1.1)
    ax.axhline(RU_THRESHOLD, color='red', linestyle='--', linewidth=1.5, label=f'License Threshold ({RU_THRESHOLD})')
    ax.legend()
# Rotate labels without overriding tick locator to avoid warnings
import matplotlib.pyplot as _plt
_plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_ylabel('RUs')
ax.set_title('License RU Consumption (Rounded Up)')
plt.tight_layout()
ru_png = out_dir / 'ru_breakdown.png'
plt.savefig(ru_png, dpi=150)
print('Saved:', ru_png)
plt.show()
